### **Chat Completion and Text Embedding using Azure OpenAI**
This notebook explores the OpenAI language models for chat completion and text embedding tasks

**STEP 0**: Install packages

In [1]:
# Install packages
%pip install openai
%pip install numpy
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


**STEP 1**: Import packages

In [2]:
# Import packages
import os
import time
from openai import AzureOpenAI
import numpy as np
from dotenv import load_dotenv

**STEP 2**: Create Azure OpenAI client

In [3]:
# Load environment variables from the .env file
load_dotenv()

True

In [4]:
api_key = os.getenv('AZUREOPENAI_API_KEY')
api_version = os.getenv('AZUREOPENAI_API_VERSION')
azure_endpoint = os.getenv('AZUREOPENAI_API_ENDPOINT')

In [5]:
# Create Azure OpenAI client
# Make sure the environment variables are created
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint = azure_endpoint
    )

**STEP 3**: Define chat completion and text embedding functions

In [6]:
# Define chat completion function
def completeChat(prompt, style, client, model="gpt-35-turbo"):
    # Execute API call
    result = client.chat.completions.create(
        model=model,
        messages= [
            {
                "role": "system",
                "content": style,
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        max_tokens=800,
        temperature=0.2,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False
    )

    # Extract the response
    response = result.choices[0].message.content

    return response

In [7]:
# Define text embedding function
def embedText(text, client, model="text-embedding-ada-002"):
    # Execute API call
    result = client.embeddings.create(
        model=model,
        input=text
    )

    # Extract and normalize the embeddings
    embedding = np.array(result.data[0].embedding)
    embedding /= np.linalg.norm(embedding)

    return embedding

**STEP 4**: Create prompt

In [8]:
# Create prompt
style = "You are a helpful Digital Signal Processing expert."
first_topic = "Particle Filtering"
second_topic = "Particle Flow Filtering"
prompt = f"""
What are the differences between {first_topic} and {second_topic}?
Please provide a detailed comparison including:
- Common points
- Main differences


Write a little python code snippet with examples for both methods.
"""

**STEP 5**: Execute chat completion task

In [9]:
# Execute chat completion
start_time = time.time()
completion_model = "gpt-35-turbo" # Make sure it is deployed on Azure AI Studio
response = completeChat(prompt, style, client, completion_model)
elapsed_time = time.time() - start_time

# Print the response
print(response)
print(f"Time taken: {elapsed_time:.2f} seconds")

Particle Filtering and Particle Flow Filtering are both methods used for state estimation in dynamic systems. Here's a detailed comparison of the two methods:

Common Points:
1. Both methods are based on the concept of using a set of particles to represent the state of the system and updating their weights based on measurements.
2. They are both used for non-linear and non-Gaussian state estimation problems.
3. Both methods are iterative and involve a prediction step followed by a measurement update step.

Main Differences:
1. Particle Filtering:
   - In particle filtering, the particles are propagated through the system using the system dynamics model, and their weights are updated based on the likelihood of the measurements given the predicted state.
   - It is a sequential Monte Carlo method and is also known as Sequential Importance Sampling (SIS).
   - Particle filtering can suffer from sample degeneracy and particle impoverishment, especially in high-dimensional state spaces.

2.

**STEP 6**: Execute text embedding task

In [10]:
# Execute text emebdding on response from chat completion
start_time = time.time()
embedding_model = "text-embedding-ada-002" # Make sure it is deployed on Azure AI Studio
embedding = embedText(response, client, embedding_model)
elapsed_time = time.time() - start_time

# Print the embeddings
print(embedding)
print(embedding.shape)
print(np.linalg.norm(embedding))
print(f"Time taken: {elapsed_time:.2f} seconds")

[-0.01215332  0.02305597 -0.03151163 ... -0.01013457 -0.01059677
 -0.04551378]
(1536,)
1.0
Time taken: 0.12 seconds


**STEP 7**: Compute similarity score between query and response embeddings

In [11]:
# Compare chat completion response to a query
start_time = time.time()
first_keyword = "Bayesian Filtering"
second_keyword = "State Estimation"
query = f"Does the response talk about {first_keyword} and {second_keyword}?"
query_embedding = embedText(query, client, embedding_model)
similarity = np.dot(embedding, query_embedding) # Compute cosine similarity via inner product
elapsed_time = time.time() - start_time

# Print the similarity result
print(similarity)
print(f"Time taken: {elapsed_time:.2f} seconds")

0.7988360467541769
Time taken: 0.19 seconds
